# Digital Dynamical Decoupling (DDD) with Mitiq

### In this section we will complete the following steps:

   #### 1. Set up an example circuit in a Mitiq-supported frontend
   
   #### 2. Insert DDD sequences in the circuit
          a. Insert a built-in DDD sequence in the circuit
          b. Define a custom rule for inserting DDD sequences
          
   #### 3. Use Mitiq's top level function `mitiq.ddd.execute_with_ddd()` to check that the results are improved with mitigation.

## Step 1: Set up circuit
A quantum circuit can be visualized as a 2D grid where the horizontal axis represents discrete time steps (often called moments) and the vertical axis represents the qubits of the circuit. 
Each gate occupies one or more grid cells, depending on the number of qubits it acts on.

This 2D grid is essentially what we get each time we print a circuit out. 

A slack window is an horizontal and contiguous sequence of zeros in the mask matrix, corresponding to a qubit which is idling for a finite amount of time. 
Find the slack windows in the circuit below:

In [1]:
from cirq import Circuit, X, H, SWAP, LineQubit

qreg = LineQubit.range(8)
x_layer = Circuit(X.on_each(qreg))
cnots_layer = Circuit(SWAP.on(q, q + 1) for q in qreg[:-1])
circuit = x_layer + cnots_layer + x_layer
circuit

0: ───X───×───────────────────────────X───
          │
1: ───X───×───×───────────────────────X───
              │
2: ───X───────×───×───────────────────X───
                  │
3: ───X───────────×───×───────────────X───
                      │
4: ───X───────────────×───×───────────X───
                          │
5: ───X───────────────────×───×───────X───
                              │
6: ───X───────────────────────×───×───X───
                                  │
7: ───X───────────────────────────×───X───

## Step 2: Insert DDD sequences in the circuit

The DDD error mitigation technique consists of filling the slack windows of a circuit with DDD gate sequences.
This can be directly achieved via the function `insert_ddd_sequences()` function.

#### Common dynamical decoupling sequences are arrays of (evenly spaced) Pauli gates. 
In particular, Mitiq has built-in:

XX sequences, typically appropriate for mitigating (time-correlated) dephasing noise:

In [2]:
from mitiq import ddd
xx_rule = ddd.rules.xx
circuit_with_ddd_xx = ddd.insert_ddd_sequences(circuit, rule=xx_rule)
circuit_with_ddd_xx

0: ───X───×───I───I───X───I───X───I───X───
          │
1: ───X───×───×───I───X───I───X───I───X───
              │
2: ───X───────×───×───I───X───X───I───X───
                  │
3: ───X───X───X───×───×───I───X───X───X───
                      │
4: ───X───I───X───X───×───×───X───X───X───
                          │
5: ───X───I───X───X───I───×───×───────X───
                              │
6: ───X───I───X───I───X───I───×───×───X───
                                  │
7: ───X───I───I───X───I───X───I───×───X───

YY sequences, typically appropriate for mitigating (time-correlated) amplitude damping noise:

In [3]:
from mitiq import ddd
yy_rule = ddd.rules.yy
circuit_with_ddd_yy = ddd.insert_ddd_sequences(circuit, rule=yy_rule)
circuit_with_ddd_yy

0: ───X───×───I───I───Y───I───Y───I───X───
          │
1: ───X───×───×───I───Y───I───Y───I───X───
              │
2: ───X───────×───×───I───Y───Y───I───X───
                  │
3: ───X───Y───Y───×───×───I───Y───Y───X───
                      │
4: ───X───I───Y───Y───×───×───Y───Y───X───
                          │
5: ───X───I───Y───Y───I───×───×───────X───
                              │
6: ───X───I───Y───I───Y───I───×───×───X───
                                  │
7: ───X───I───I───Y───I───Y───I───×───X───

XYXY sequences: typically appropriate for mitigating generic single-qubit noise.
Let's use the XYXY rule for our workflow. 
You can experiment with other built-in sequences or define your own.

In [4]:
from mitiq import ddd
xyxy_rule = ddd.rules.xyxy
circuit_with_ddd = ddd.insert_ddd_sequences(circuit, rule=xyxy_rule)
circuit_with_ddd

0: ───X───×───I───X───Y───X───Y───I───X───
          │
1: ───X───×───×───I───X───Y───X───Y───X───
              │
2: ───X───────×───×───X───Y───X───Y───X───
                  │
3: ───X───────────×───×───────────────X───
                      │
4: ───X───────────────×───×───────────X───
                          │
5: ───X───X───Y───X───Y───×───×───────X───
                              │
6: ───X───I───X───Y───X───Y───×───×───X───
                                  │
7: ───X───I───X───Y───X───Y───I───×───X───

### Define a custom rule for inserting DDD sequences

In [5]:
import numpy as np 

def custom_rule(slack_length: int) -> Circuit:
    q = LineQubit(0)
    if slack_length == 2:
        sequence = Circuit([H(q), H(q)])
    elif slack_length == 4:
        sequence = Circuit([H(q), H(q), H(q), H(q)])
    else:
        sequence = Circuit()
    return sequence

# Test
print(custom_rule(2))
print(custom_rule(4))

0: ───H───H───
0: ───H───H───H───H───


## Step 3. Use Mitiq's top level function `mitiq.ddd.execute_with_ddd()` and check that the results are improved with mitigation.

Define an executor function to evaluate expectation values.

In [6]:
import numpy as np
from cirq import DensityMatrixSimulator, amplitude_damp
from mitiq.interface import convert_to_mitiq

def execute(circuit, noise_level=0.01):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with amplitude damping noise.
    """
    mitiq_circuit, _ = convert_to_mitiq(circuit)
    noisy_circuit = mitiq_circuit.with_noise(amplitude_damp(gamma=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

Compute the noisy (unmitigated) and ideal expectation values.

In [7]:
# Compute the expectation value of the |0><0| observable.
noisy_value = execute(circuit)
ideal_value = execute(circuit, noise_level=0.0)
print(f"Error without mitigation: {abs(ideal_value - noisy_value) :.3}")

Error without mitigation: 0.471


In [8]:
# Ideal result
execute(circuit, noise_level=0)

# Unmitigated result
execute(circuit)


mitigated = ddd.execute_with_ddd(circuit, execute, rule=xyxy_rule)
print(f"Error with mitigation: {abs(ideal_value - mitigated) :.3}")


Error with mitigation: 0.402
